In [ ]:
import os
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool
from langchain.graph import GraphExecutor
from langchain.visualization import visualize_graph

# Define the LLM (Large Language Model) you will use
llm = OpenAI(model="text-davinci-003", temperature=0.5)

# Step 1: Define a custom tool to fetch data
def fetch_data(query: str) -> str:
    # This is a placeholder for fetching data, e.g., from a database or API.
    return f"Fetched data for the query: {query}"

fetch_tool = Tool(
    name="FetchTool",
    func=fetch_data,
    description="Tool to fetch data based on the query"
)

# Step 2: Define the first chain to process the query and fetch data
prompt_1 = PromptTemplate(input_variables=["query"], template="What do you know about {query}?")
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

# Step 3: Define the second chain to summarize the fetched data
prompt_2 = PromptTemplate(input_variables=["data"], template="Summarize this data: {data}")
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

# Step 4: Add memory to the chain for keeping track of previous context (optional)
memory = ConversationBufferMemory()
chain_1_with_memory = LLMChain(llm=llm, prompt=prompt_1, memory=memory)
chain_2_with_memory = LLMChain(llm=llm, prompt=prompt_2, memory=memory)

# Step 5: Create a graph of the agent architecture (flow of chains)
graph = {
    "input": ["fetch_tool"],
    "fetch_tool": ["chain_1_with_memory"],
    "chain_1_with_memory": ["chain_2_with_memory"],
    "chain_2_with_memory": ["output"]
}

# Step 6: Execute the graph using GraphExecutor
executor = GraphExecutor(graph=graph, tools={"fetch_tool": fetch_tool, "chain_1_with_memory": chain_1_with_memory, "chain_2_with_memory": chain_2_with_memory})

# Step 7: Define input data for testing
input_data = {"query": "LangChain"}

# Run the agent architecture
output = executor.run(input_data)

# Step 8: Print the output result
print(f"Final Output: {output}")

# Step 9: Visualize the agent's graph architecture
visualize_graph(graph)
